In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing , ensemble
from sklearn.svm import SVR
from math import sqrt
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier
import torch

In [2]:
# 原始資料
'''
columns = ['Age', 'Workclass', 'fnlwgt', 'education'
    , 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'salary']
'''
columns = ['Age','Workclass','fnlgwt','Education'
    ,'EdNum','MaritalStatus','Occupation','Relationship','Race','Sex','CapitalGain','CapitalLoss',
           'HoursPerWeek','Country','Income']


df_train_set = pd.read_csv('datasets/adult.data', names=columns, na_values=' ?')
df_test_set = pd.read_csv('datasets/adult.test', names=columns, skiprows=1, na_values=' ?')

In [3]:
df_train_set.drop('fnlgwt', axis=1, inplace=True)
df_test_set.drop('fnlgwt', axis=1, inplace=True)
df_train_set = df_train_set.dropna()
df_test_set = df_test_set.dropna()

In [4]:
df_train_set.drop(["Country", "Education"], axis=1, inplace=True) 
df_test_set.drop(["Country", "Education"], axis=1, inplace=True)


In [5]:
colnames = list(df_train_set.columns) 
colnames.remove('Age')
colnames.remove('EdNum')
colnames = ['AgeGroup', 'EduGroup']+colnames

labels = ["{0}-{1}".format(i, i+9) for i in range(0,100,10)]
# https://www.runoob.com/python/att-string-format.html 利用format快速创建字符型列表
df_train_set['AgeGroup'] = pd.cut(df_train_set.Age, range(0,101,10), right = False, labels = labels)
# https://www.cnblogs.com/sench/p/10128216.html 用来把一组数据分割成离散的区间 
df_test_set['AgeGroup'] = pd.cut(df_test_set.Age, range(0,101,10), right = False, labels = labels) 

labels = ["{0}-{1}".format(i,i+4) for i in range(0,20,5)]    
df_train_set['EduGroup'] = pd.cut(df_train_set.EdNum, range(0,21,5), right = False, labels = labels)   
df_test_set['EduGroup'] = pd.cut(df_test_set.EdNum, range(0,21,5), right = False, labels = labels)

df_train_set = df_train_set[colnames] # 仅提取包含colnames中的列，且按照colnames排序
df_test_set = df_test_set[colnames]


In [6]:
from sklearn_pandas import DataFrameMapper
mapper = DataFrameMapper([('AgeGroup', LabelEncoder()),('EduGroup', LabelEncoder()),
                          ('Workclass', LabelEncoder()),('MaritalStatus', LabelEncoder()),
                          ('Occupation', LabelEncoder()),('Relationship', LabelEncoder()),
                          ('Race', LabelEncoder()),('Sex', LabelEncoder()),
                          ('Income', LabelEncoder())], df_out=True, default=None)

cols = list(df_train_set.columns)
cols.remove('Income')
cols = cols[:-3]+['Income']+cols[-3:]  

df_train = mapper.fit_transform(df_train_set.copy())
df_train.columns = cols
df_test = mapper.fit_transform(df_test_set.copy())
df_test.columns = cols

train_minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
train_data_minmax = train_minmax.fit_transform(df_train[cols])
test_data_minmax = train_minmax.transform(df_test[cols])

df_train = pd.DataFrame(train_data_minmax,columns =cols)
df_test = pd.DataFrame(test_data_minmax,columns =cols)

cols.remove('HoursPerWeek')

In [7]:
x_train, y_train = df_train[cols].values, df_train['HoursPerWeek'].values
x_test, y_test = df_test[cols].values, df_test['HoursPerWeek'].values

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2,random_state=42)
df_train.head(10)


,AgeGroup,EduGroup,Workclass,MaritalStatus,Occupation,Relationship,Race,Sex,Income,CapitalGain,CapitalLoss,HoursPerWeek
0,0.250,0.333333,0.833333,0.666667,0.000000,0.2,1.0,1.0,0.0,0.021740,0.0,0.397959
1,0.500,0.333333,0.666667,0.333333,0.230769,0.0,1.0,1.0,0.0,0.000000,0.0,0.122449
2,0.250,1.000000,0.333333,0.000000,0.384615,0.2,1.0,1.0,0.0,0.000000,0.0,0.397959
3,0.500,1.000000,0.333333,0.333333,0.384615,0.0,0.5,1.0,0.0,0.000000,0.0,0.397959
4,0.125,0.333333,0.333333,0.333333,0.692308,1.0,0.5,0.0,0.0,0.000000,0.0,0.397959
5,0.250,0.333333,0.333333,0.333333,0.230769,1.0,1.0,0.0,0.0,0.000000,0.0,0.397959
6,0.375,1.000000,0.333333,0.500000,0.538462,0.2,0.5,0.0,0.0,0.000000,0.0,0.153061
7,0.500,1.000000,0.666667,0.333333,0.230769,0.0,1.0,1.0,1.0,0.000000,0.0,0.448980
8,0.250,0.333333,0.333333,0.666667,0.692308,0.2,1.0,0.0,1.0,0.140841,0.0,0.500000
9,0.375,0.333333,0.333333,0.333333,0.230769,0.0,1.0,1.0,1.0,0.051781,0.0,0.397959


In [8]:
def mape_loss_func(labels,preds):
    mask=labels!=0
    return np.fabs((preds[mask]-labels[mask])/labels[mask]).mean()

In [9]:
forest = ensemble.RandomForestRegressor(random_state=42,max_depth=10,n_estimators=90)
forest_fit = forest.fit(x_train, y_train)
test_y_predicted = forest_fit.predict(x_validation)

print("RMSE : %.4g" % metrics.mean_squared_error(y_validation, test_y_predicted,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_validation, test_y_predicted))


RMSE : 0.1052
MAPE : 0.2802


In [10]:
forest = ensemble.RandomForestRegressor(random_state=42,max_depth=10,n_estimators=90)
forest_fit = forest.fit(x_train, y_train)
test_y_predicted = forest_fit.predict(x_test)

print("RMSE : %.4g" % metrics.mean_squared_error(y_test, test_y_predicted,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_test, test_y_predicted))

RMSE : 0.1075
MAPE : 0.2859
